In [ ]:
import pandas as pd
import numpy as np
import scipy
from scipy import ndimage
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from IPython.display import clear_output

import tensorflow as tf

import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.utils.np_utils import to_categorical  
from keras import regularizers
from keras.utils.np_utils import to_categorical

from constants import num_class

## Callbacks

In [ ]:
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.acc = []
        self.val_losses = []
        self.val_acc = []
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        f, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=False)
        self.ax1 = ax1
        self.ax2 = ax2
    
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.acc.append(logs.get('categorical_accuracy'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_acc.append(logs.get('val_categorical_accuracy'))
        self.i += 1
        
        clear_output(wait=True)
        self.ax1.plot(self.x, self.losses, label="loss")
        self.ax1.plot(self.x, self.val_losses, label="val loss")
        self.ax1.legend()
        self.ax2.plot(self.x, self.acc, label="accuracy")
        self.ax2.plot(self.x, self.val_acc, label="val accuracy")
        self.ax2.legend()
        plt.show()
        
        for i in range(self.i):
            print('Epoch ' + str(i+1))
            print('-----------------------')
            print('- Loss:', self.losses[i])
            print('- Accuracy:', self.acc[i])
            print('- Validation loss:', self.val_losses[i])
            print('- Validation accuracy:', self.val_acc[i])
            print(' ')
            
checkpoint = keras.callbacks.ModelCheckpoint(
    './convnet_weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=False, 
    save_weights_only=True, 
    mode='auto', 
    period=1,
)

## Data

In [ ]:
train_labels = to_categorical(np.load('./features/res_net/classes_train.npy'))
dev_labels = to_categorical(np.load('./features/res_net/classes_dev.npy'))

In [ ]:
train_data_1 = np.load('./features/res_net/features_train.npy')
dev_data_1 = np.load('./features/res_net/features_dev.npy')

In [ ]:
train_data_2 = np.load('./features/inception/features_train.npy')
dev_data_2 = np.load('./features/inception/features_dev.npy')

In [ ]:
train_data_3 = np.load('./features/xception/features_train.npy')
dev_data_3 = np.load('./features/xception/features_dev.npy')

In [ ]:
train_data = np.concatenate([train_data_1, train_data_2, train_data_3], axis=1)
dev_data = np.concatenate([dev_data_1, dev_data_2, dev_data_3], axis=1)
train_data_1 = None
train_data_2 = None
train_data_3 = None
dev_data_1 = None
dev_data_2 = None
dev_data_3 = None

## Top Model

In [ ]:
def model(train_data, dev_data, lr = 0.00001):
    inputs = Input(shape=(train_data.shape[1:]))
    X = Dropout(0.5)(inputs)
    X = Dense(4096, activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(2048, activation='relu')(X)
    X = Dense(2048, activation='relu')(X)
    X = Dense(1024, activation='relu')(X)

    outputs = Dense(
        num_class, 
        activation='softmax', 
        #activity_regularizer=regularizers.l2(100.)
    )(X)

    top_model = Model(inputs=inputs, outputs=outputs)

    top_model.compile(optimizer=tf.train.AdamOptimizer(lr),
                  loss='categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    
    return top_model

## Concatenation

In [ ]:
plot_losses = PlotLosses()
top_model = model(train_data, dev_data, 0.00001)

In [ ]:
#top_model.load_weights('./model_1/weights.03-0.53.hdf5')

top_model.fit(
    train_data, train_labels,
    batch_size=128,
    epochs=50,
    callbacks=[plot_losses, checkpoint], 
    validation_data=(dev_data, dev_labels),
)

## Averaging

In [ ]:
plot_losses = PlotLosses()
top_model_1 = model(train_data_1, dev_data_1, 0.00001)

In [ ]:
#top_model.load_weights('./model_1/weights.50-10.92.hdf5')

top_model_1.fit(
    train_data_1, train_labels,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[plot_losses, checkpoint], 
    validation_data=(dev_data_1, dev_labels),
)

In [ ]:
plot_losses = PlotLosses()
top_model_2 = model(train_data_2, dev_data_2, 0.00001)

In [ ]:
#top_model.load_weights('./model_1/weights.50-10.92.hdf5')

top_model_2.fit(
    train_data_2, train_labels,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[plot_losses, checkpoint], 
    validation_data=(dev_data_2, dev_labels),
)

### Predictions

In [ ]:
real_classes = np.load('./data/bottleneck_features/res_net/classes.npy')
real_classes

In [ ]:
top_model_1.load_weights('./best_models/weights.13-2.82.hdf5')
predictions_1 = top_model_1.predict(dev_data_1)

In [ ]:
top_model_2.load_weights('./best_models/weights.13-2.82.hdf5')
predictions_2 = top_model_2.predict(dev_data_2)

In [ ]:
predictions = np.stack([predictions_1, predictions_2], axis=0)
#predictions = np.max(predictions, axis=0)
predictions = np.mean(predictions, axis=0)

In [ ]:
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes

In [ ]:
filenames = np.load('./data/bottleneck_features/vgg16/filenames.npy')

In [ ]:
predictions_df = pd.DataFrame({
    "filenames": filenames,
    "real": real_classes,
    "predictions": predicted_classes
})

In [ ]:
wrong_answers = predictions_df[predictions_df['real'] != predictions_df['predictions']]
wrong_answers.shape

In [ ]:
wrong_answers['real'].value_counts()[0:10]

In [ ]:
predictions_df[predictions_df['real'] == 71]['real'].value_counts()

In [ ]:
for filename in wrong_answers['filenames'][0:10]:
    image = np.array(ndimage.imread('./data/dev_dataset/' + filename, flatten=False))
    plt.figure()
    plt.imshow(image)